# Setup

Turn on autoreload. All modules will be reloaded before excecuting each cell. Even objects that already exist will receive the new method defintions.

In [ ]:
# RUN THIS
%load_ext autoreload
%autoreload 2

Import stuff.

In [ ]:
# RUN THIS
import logging
import sys
import os
import atexit

from cirrus import automate, GridSearch, lr, graph, configuration

Configure logging so that debug-level log messages are printed.

In [ ]:
# RUN THIS
log = logging.getLogger("cirrus")
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))

# Done on build environment change

__Make build image__

Create the build image, which is an AMI with the right environment set up for building Cirrus.

In [ ]:
automate.make_build_image("cirrus_build_image", True)

# Done on release

__Make executables__

Build Cirrus and publish its executables.

The `cirrus-public` S3 bucket that we publish to is owned by Joao. This cell will fail if run under any account other than his.

In [ ]:
# Start an instance to be used for making the executables.
region = configuration.config()["aws"]["region"]
instance = automate.Instance("cirrus_instance_for_executables", ami_name="cirrus_build_image",
    disk_size=32, typ="m5.2xlarge", username="ec2-user")
instance.start()

In [ ]:
# Make the executables.
automate.make_executables("s3://cirrus-public/0/executables", instance)

In [ ]:
instance.public_ip()

In [ ]:
# Terminate the instance.
instance.cleanup()

__Make server image__

Create and publish the server image, which is an AMI that comes ready with Cirrus' executables and the proper execution environment. Each Cirrus installation will use this image to create instances in the user's AWS account.

In [ ]:
# RUN THIS
# It is necessary to hardcode the Ubuntu 16.04 LTS AMI for each region we support.
# Update using by searching "us-west aws 16.04 ssd" on https://cloud-images.ubuntu.com/locator/.
UBUNTU_AMIS = {
    "us-west-1": "ami-0f5113ce7686689e9",
    "us-west-2": "ami-01e0cf6e025c036e4"
}

In [ ]:
# RUN THIS
# Start an instance to be used for making the image.
region = configuration.config()["aws"]["region"]
instance = automate.Instance("cirrus_instance_for_server_image", ami_id=UBUNTU_AMIS[region],
    disk_size=32, typ="m5.2xlarge", username="ubuntu")
instance.start()

In [ ]:
# RUN THIS
# Make the image.
automate.make_server_image("cirrus_server_image", "s3://cirrus-public/0/executables", instance)

In [ ]:
# RUN THIS
# Terminate the instance.
instance.cleanup()

__Make Lambda package__

Create and publish the Lambda package, which is a ZIP archive containing the code to be executed by the worker Lambda function. Each Cirrus installation will use this package to create a Lambda function in the user's AWS account.

The `cirrus-public` S3 bucket that we publish to is owned by Joao. This cell will fail if run under any account other than his.

In [ ]:
automate.make_lambda_package(
    "s3://cirrus-public/0/lambda-package",
    "s3://cirrus-public/0/executables"
)

# Done by user

Make the lambda.

In [ ]:
automate.make_lambda("cirrus_worker", "s3://cirrus-public/0/lambda-package", 100)

Make the instance for the parameter servers.

In [ ]:
# RUN THIS
instance_for_ps = automate.Instance(
    "cirrus_instance_for_ps",
    ami_name="cirrus_server_image",
    disk_size=32,
    typ="m5.2xlarge",
    username="ubuntu"
)
instance_for_ps.start()

Define the logistic regression configuration.

In [ ]:
# RUN THIS
basic_params = {
    'n_workers': 16,
    'n_ps': 1,
    'lambda_size': 128,
    'dataset': "criteo-kaggle-19b",
    'learning_rate': 0.001,
    'epsilon': 0.0001,
    'progress_callback': None,
    'timeout': 60,
    'opt_method': 'adagrad',
    'minibatch_size': 200,
    'model_bits': 19,
    'train_set': (0,6),
    'test_set': (7,8)
}

Start a grid search.

In [ ]:
# RUN THIS
gs = GridSearch.GridSearch(
    task=lr.LogisticRegression,
    param_base=basic_params,
    hyper_vars=["learning_rate"],
    hyper_params=[[0.001, 0.01, 0.1, 1]],
    instances=[instance_for_ps]
)
gs.set_threads(1)
gs.run(UI=False)

In [ ]:
instance_for_ps.cleanup()

View the progress of the experiment.

In [ ]:
graph.display_dash()

# Utilities

Write the SSH key for the instance to a file.

In [ ]:
with open("key.pem", "w+") as f:
    f.write(instance_for_ps.private_key())

View the instance's public IP address.

In [ ]:
instance_for_ps.public_ip()

View the output of the parameter server of the first experiment.

In [ ]:
# RUN THIS
gs.cirrus_objs[0].ps.error_output()

In [ ]:
# RUN THIS
gs.cirrus_objs[0].ps.ps_output()

In [ ]:
gs.kill_all()

Create a new SSH connection to the instance. Useful when the connection times out while you're aware from the computer.

In [ ]:
instance_for_ps._connect_ssh()

Do cleanup. Normally this is called on exit, but Jupyter does not reliably allow this to complete.

In [ ]:
atexit._run_exitfuncs()